In [1]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7e92e823-5372-4b8a-8348-131a70af7993)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 4, Finished, Available, Finished)

In [3]:
#Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 5, Finished, Available, Finished)

In [4]:
#Apply the model to our dataframe
result = model.transform(df)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 6, Finished, Available, Finished)

In [5]:
display(result)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d24ada3a-1321-4775-b5a3-d3811464b46e)

In [6]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("sentiment", col("response.documents.sentiment"))

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 8, Finished, Available, Finished)

In [7]:
display(sentiment_df)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a9c215de-122d-4691-bf9a-73c08116bc49)

In [8]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 10, Finished, Available, Finished)

In [9]:
display(sentiment_df_final)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e16bde90-a4e8-4aa5-922a-cf246edab6e4)

In [10]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 12, Finished, Available, Finished)

In [13]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_sentiment_analysis'

    sentiment_df_final.write.format("delta").saveAsTable(table_name)

except AnalysisException:

    print("Table Already Exists")

    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")

    spark.sql(f"""  MERGE INTO {table_name} target_table
                    USING vw_sentiment_df_final source_view

                    ON source_view.url = target_table.url
                    
                    WHEN MATCHED AND 
                    source_view.title <> target_table.title OR
                    source_view.description <> target_table.description OR
                    source_view.category <> target_table.category OR
                    source_view.image <> target_table.image OR
                    source_view.provider <> target_table.provider OR
                    source_view.datePublished <> target_table.datePublished   

                    THEN UPDATE SET *

                    WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 15, Finished, Available, Finished)

Table Already Exists


In [12]:
display(sentiment_df_final)

StatementMeta(, 99f5cb85-074a-4e25-bc70-84f321209e26, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a51093f2-3dcb-4a22-a104-9a143ec9f3d9)

In [ ]:
%%sql

SELECT count(*) from bing_lake_db.tbl_sentiment_analysis